In [18]:
import nltk
from nltk.corpus import stopwords
import copy
import string
import unicodedata
import numpy as np
import pandas as pd
from itertools import chain
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from scipy.sparse import csr_matrix, lil_matrix
from time import perf_counter
from sklearn import preprocessing
from scipy.linalg import fractional_matrix_power, funm
from scipy.io import arff
from math import sqrt
from numba import jit

''' 
    Create a co-ocurrence matrix and map each word to an unique id.
    Input: Matrix of documents by row
    Output: An co-ocurrence matrix and a dict{word:id}
'''
def create_co_occurences_matrix(X, words):
    word_to_id = dict(zip(words, range(len(words))))
    X[X>0] = 1
    return csr_matrix(X.T.dot(X)), word_to_id

def load_from_arff(filename, class_column_name):
    data, meta = arff.loadarff(open(filename, 'r'))

    X = pd.DataFrame(data)
    y = X[class_column_name]
    
    X.drop(columns=[class_column_name], inplace=True)
    X.sort_index(axis=1, inplace=True)
    
    return X.values, y, sorted(meta.names()[:-1])

def split_balanced(data, target, test_size=0.2):

    classes = np.unique(target)
    # can give test_size as fraction of input data size of number of samples
    if test_size<1:
        n_test = np.round(len(target)*test_size)
    else:
        n_test = test_size
    n_train = max(0,len(target)-n_test)
    n_train_per_class = max(1,int(np.floor(n_train/len(classes))))
    n_test_per_class = max(1,int(np.floor(n_test/len(classes))))

    ixs = []
    for cl in classes:
        if (n_train_per_class+n_test_per_class) > np.sum(target==cl):
            # if data has too few samples for this class, do upsampling
            # split the data to training and testing before sampling so data points won't be
            #  shared among training and test data
            splitix = int(np.ceil(n_train_per_class/(n_train_per_class+n_test_per_class)*np.sum(target==cl)))
            ixs.append(np.r_[np.random.choice(np.nonzero(target==cl)[0][:splitix], n_train_per_class),
                np.random.choice(np.nonzero(target==cl)[0][splitix:], n_test_per_class)])
        else:
            ixs.append(np.random.choice(np.nonzero(target==cl)[0], n_train_per_class+n_test_per_class,
                replace=False))

    # take same num of samples from all classes
    ix_train = np.concatenate([x[:n_train_per_class] for x in ixs])
    ix_test = np.concatenate([x[n_train_per_class:(n_train_per_class+n_test_per_class)] for x in ixs])

    X_train = data[ix_train,:]
    X_test = data[ix_test,:]
    y_train = target[ix_train]
    y_test = target[ix_test]

    return X_train, X_test, y_train, y_test

In [19]:
start_execution = perf_counter()

X, y, words = load_from_arff('../data/CSTR.arff', 'class_atr')
y, levels = pd.factorize(y)

total_docs = len(X)

#df = load_from_csv('small_news_.csv')
#X = pre_process(df['0'].values)
#y = df['class']

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=len(X) - len(np.unique(y)), random_state=42, stratify=y)
X_train, X_test, y_train, y_test = split_balanced(X, y, test_size=len(X) - (2*len(np.unique(y))))

X = csr_matrix(np.concatenate((X_train,X_test)))
y = np.concatenate((y_train,y_test))

print(len(y_test), len(y_train), y_train)

words_cooc_matrix, word_to_id = create_co_occurences_matrix(X, words)

n_labeled = len(y_train)

# Binarize labels in a one-vs-all fashion
lb = preprocessing.LabelBinarizer()
lb.fit(y_train)
Y_bin = lb.transform(y_train)
Y_input = np.concatenate((Y_bin, np.zeros((y.size-Y_bin.shape[0], Y_bin.shape[1]))))

print("Execution time: ", perf_counter() - start_execution)

288 8 [0 0 1 1 2 2 3 3]
Execution time:  0.355702112000003


In [20]:
'''
    Construct a contingency matrix
'''
@jit
def full_contingency_matrix(cooc_matrix, total_docs):
    p_11 = (cooc_matrix/total_docs)
    p_10 = (csr_matrix.diagonal(cooc_matrix)[:, np.newaxis] - cooc_matrix)/total_docs
    p_01 = (csr_matrix.diagonal(cooc_matrix) - cooc_matrix)/total_docs
    p_00 = 1-(p_11+p_10+p_01)
    
    return np.array([p_11, p_10, p_01, p_00])

In [21]:
start_execution = perf_counter()
A = full_contingency_matrix(words_cooc_matrix, total_docs)
print("Full contingency matrix time: ", perf_counter() - start_execution)

Full contingency matrix time:  0.23973525799999607


In [80]:
@jit             
def support(cm):
    return cm[0].A

@jit             
def piatetsky_shapiro(cm):
    p_11 = cm[0]
    return (p_11 - (p_11.diagonal()*p_11.diagonal()[:,np.newaxis]))

@jit
def yule(cm):
    p_11 = cm[0]
    p_10 = cm[1]
    p_01 = cm[2]
    p_00 = cm[3]
    
    part1 = p_11*p_00
    part2 = p_10*p_01
    
    numerator = (part1 - part2)
    denominator = (part1 + part2)
    
    return np.divide(numerator,denominator,where=denominator!=0)

@jit
def mutual_information(cm):
    p_11 = cm[0].A
    p_10 = cm[1].A
    p_01 = cm[2].A
    p_00 = cm[3].A
    
    p_1_p_1 = (np.diag(p_11)*np.diag(p_11)[:,np.newaxis])
    p_1_p_0 = (np.diag(p_00)*np.diag(p_11)[:,np.newaxis])
    p_0_p_1 = (np.diag(p_11)*np.diag(p_00)[:,np.newaxis])
    p_0_p_0 = (np.diag(p_00)*np.diag(p_00)[:,np.newaxis])
    '''
    p_1_p_1 = (p_11.diagonal()*p_11.diagonal()[:,np.newaxis])
    p_1_p_0 = (p_00.diagonal()*p_11.diagonal()[:,np.newaxis])
    p_0_p_1 = (p_11.diagonal()*p_00.diagonal()[:,np.newaxis].T)
    p_0_p_0 = (p_00.diagonal()*p_00.diagonal()[:,np.newaxis].T)
    '''
    p1 = np.divide(p_11,p_1_p_1,where=p_1_p_1!=0)
    p2 = np.divide(p_10,p_1_p_0,where=p_1_p_0!=0)
    p3 = np.divide(p_01,p_0_p_1,where=p_0_p_1!=0)
    p4 = np.divide(p_00,p_0_p_0,where=p_0_p_0!=0)
    
    return  p_11*np.log2(p1, where=p1>0) +\
            p_10*np.log2(p2, where=p2>0) +\
            p_01*np.log2(p3, where=p3>0) +\
            p_00*np.log2(p4, where=p4>0)

@jit
def kappa(cm):
    p_11 = cm[0].A
    p_00 = cm[3].A
    
    p_1_p_1 = (np.diag(p_11)*np.diag(p_11)[:,np.newaxis])
    p_0_p_0 = (np.diag(p_00)*np.diag(p_00)[:,np.newaxis])
    
    aux = p_1_p_1 - p_0_p_0
    
    num = (p_11 + p_00 - aux)
    den = 1 - (aux)
    
    return np.divide(num,den,where=den!=0)
    
@jit
def calculate_s(W):
    S = np.zeros_like(W)
    d = np.sum(W, axis=1)

    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            if d[i] == 0 or d[j] == 0:
                S[i][j] = 0
            else:
                S[i][j] = W[i][j] / (sqrt(d[i]) * sqrt(d[j]))
    return S

@jit
def fractional_power(W):
    d = np.sum(W, axis=1)
    D = np.sqrt(d*d[:, np.newaxis])
    return np.divide(W,D,where=D!=0)

def mutual_knn(adjacency_matrix):    
    adj_matrix = np.minimum(adjacency_matrix, adjacency_matrix.T)

    # Solve isolated vertices problem adding an edge to its nearest neighbor
    # Performance improving needed
    for i in np.where(~adj_matrix.any(axis=1))[0]:
        rand_j = np.random.randint(0, adj_matrix.shape[1])
        min_index, min_value = min(enumerate(adj_matrix[i,:]), key=lambda x: x[1] if x[1] > 0 else float('inf'))
        adj_matrix[i, min_index] = min_value
        adj_matrix[min_index, i] = min_value
        
    return adj_matrix

def symmetric_knn(adjacency_matrix):
    return np.maximum(adjacency_matrix, adjacency_matrix.T)

# Top-K
def topK(W, k=3, kind=None):
    adj_matrix = np.zeros_like(W.A)
    max_values = np.argpartition(W, -k, axis=1)[:,-k:]
    
    for i in range(max_values.shape[0]):
        neighbors = np.array(max_values[i]).flatten()
        for j in range(neighbors.size):
            adj_matrix[i, neighbors[j]] = 1 

    if kind:
        if kind == 'mutual':
            adj_matrix = mutual_knn(adj_matrix)
        elif kind == 'symmetric':
            adj_matrix = symmetric_knn(adj_matrix)
            
    return np.multiply(adj_matrix, W)

# Epsilon-NN
def enn(W, epsilon=0.5, kind=None):
    adj_matrix = np.where(W < epsilon, 0, 1)
    
    if kind:
        if kind == 'mutual':
            adj_matrix = mutual_knn(adj_matrix)   
        elif kind == 'symmetric':
            adj_matrix = symmetric_knn(adj_matrix)
    
    return np.multiply(adj_matrix, W)
    
    
start_execution = perf_counter()

W = enn(yule(A), 3, 'symmetric')#construct_term_network(words_cooc_matrix, A, word_to_id, total_docs)

print(perf_counter() - start_execution)
print("Min", np.min(W), "Max", np.max(W))
min_max_scaler = preprocessing.MinMaxScaler()
W = min_max_scaler.fit_transform(W)
S = fractional_power(W)
print(perf_counter() - start_execution)


0.6560250620000261
Min 0.0 Max 0.0
0.8420288190000065


In [ ]:
def initialize_scores_matrix(y, dt_matrix, words_id):
    sum_freq_total = dt_matrix.sum(axis = 0)
    F = []
    
    with np.errstate(divide='ignore', invalid='ignore'):
        for j in np.unique(y):
            F.append(np.nan_to_num(np.squeeze(np.asarray(dt_matrix[np.where(y==j)].sum(axis = 0)/sum_freq_total)), copy=False))
    
    return np.array(F).T

In [ ]:
start_execution = perf_counter()
F = initialize_scores_matrix(y_train, X_train, word_to_id)
Y_input_terms = copy.deepcopy(F) # 
print("Execution time:", perf_counter() - start_execution)

In [ ]:
#@jit
import cython

@cython.wraparound(False)
@cython.boundscheck(False)
def llgc(alpha, S, F, Y_input_terms, n_iter):
    F_result = F
    for _ in range(n_iter):
        F_result = alpha*np.dot(S, F_result) + (1-alpha)*Y_input_terms
    return F_result

In [ ]:
n_iter = 400
alpha=0.1

start_execution = perf_counter()
F = llgc(alpha, S, F, Y_input_terms, n_iter)
print(perf_counter() - start_execution)

In [ ]:
def classify(F):
    Y_result = np.zeros_like(F)
    Y_result[np.arange(len(F)), F.argmax(1)] = 1
    return Y_result

def classify_docs(X, F):
    return X.dot(F)

In [ ]:
F_docs = classify_docs(X,F)
y_answer = classify(F_docs)
y_v = lb.inverse_transform(y_answer)

print('Total of documents correctly classified:',sum(y_v==y), sum(y_v==y)/y.shape[0])

In [ ]:
y_answer